# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [19]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [20]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,2.58,88,91,12.55,TF,1720412813
1,1,albany,42.6001,-73.9662,21.44,85,0,1.25,US,1720412814
2,2,adamstown,-25.0660,-130.1015,20.04,69,100,9.92,PN,1720412815
3,3,ust-nera,64.5667,143.2000,16.48,51,30,2.69,RU,1720412633
4,4,ushuaia,-54.8000,-68.3000,2.81,81,40,7.72,AR,1720412634


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [30]:
%%capture --no-display

# Configure the map plot
map_plot_all = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
map_plot_all

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [31]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df[(city_data_df['Humidity'] < 30) & (city_data_df['Cloudiness'] == 0) & (city_data_df['Wind Speed'] < 5)] 

# Drop any rows with null values
ideal_df = ideal_df.dropna(how='any')

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,phalaborwa,-23.9430,31.1411,16.60,12,0,2.82,ZA,1720412845
89,89,eagle point,42.4726,-122.8028,33.01,23,0,3.09,US,1720412789
124,124,corona de tucson,31.9654,-110.7756,35.95,25,0,1.79,US,1720412979
164,164,socorro,31.6546,-106.3033,34.55,18,0,2.57,US,1720413031
237,237,sabha,27.0377,14.4283,26.15,26,0,4.14,LY,1720413127
239,239,tahe,52.3333,124.7333,28.32,23,0,3.50,CN,1720413129
284,284,menongue,-14.6585,17.6910,6.76,26,0,0.44,AO,1720413187
313,313,beyneu,45.3167,55.2000,31.62,26,0,4.35,KZ,1720413224
326,326,spring creek,40.7266,-115.5859,24.83,18,0,2.64,US,1720413241
340,340,ghadamis,30.1337,9.5007,29.79,18,0,4.39,LY,1720413258


### Step 3: Create a new DataFrame called `hotel_df`.

In [32]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
25,phalaborwa,ZA,-23.9430,31.1411,12,
89,eagle point,US,42.4726,-122.8028,23,
124,corona de tucson,US,31.9654,-110.7756,25,
164,socorro,US,31.6546,-106.3033,18,
237,sabha,LY,27.0377,14.4283,26,
239,tahe,CN,52.3333,124.7333,23,
284,menongue,AO,-14.6585,17.6910,26,
313,beyneu,KZ,45.3167,55.2000,26,
326,spring creek,US,40.7266,-115.5859,18,
340,ghadamis,LY,30.1337,9.5007,18,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [33]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":1,
    "filter":"",
    "bias":"",
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
phalaborwa - nearest hotel: Hans Merensky Hotel & Estate
eagle point - nearest hotel: Brookside
corona de tucson - nearest hotel: No hotel found
socorro - nearest hotel: WoodSpring Suites El Paso I-10 Southeast
sabha - nearest hotel: فندق الكاظم-سبها
tahe - nearest hotel: No hotel found
menongue - nearest hotel: No hotel found
beyneu - nearest hotel: Johan yu
spring creek - nearest hotel: No hotel found
ghadamis - nearest hotel: فندق عين الفارس
nalut - nearest hotel: No hotel found
illizi - nearest hotel: بوناقة
mhamid - nearest hotel: Hotel Kasbah Azalay
murzuq - nearest hotel: فندق باريس - مرزق
langxiang - nearest hotel: No hotel found
caconda - nearest hotel: No hotel found
ouargla - nearest hotel: LYNATEL
ha'il - nearest hotel: فندق كلاسيك


,City,Country,Lat,Lng,Humidity,Hotel Name
25,phalaborwa,ZA,-23.9430,31.1411,12,Hans Merensky Hotel & Estate
89,eagle point,US,42.4726,-122.8028,23,Brookside
124,corona de tucson,US,31.9654,-110.7756,25,No hotel found
164,socorro,US,31.6546,-106.3033,18,WoodSpring Suites El Paso I-10 Southeast
237,sabha,LY,27.0377,14.4283,26,فندق الكاظم-سبها
239,tahe,CN,52.3333,124.7333,23,No hotel found
284,menongue,AO,-14.6585,17.6910,26,No hotel found
313,beyneu,KZ,45.3167,55.2000,26,Johan yu
326,spring creek,US,40.7266,-115.5859,18,No hotel found
340,ghadamis,LY,30.1337,9.5007,18,فندق عين الفارس


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [34]:
%%capture --no-display

# Configure the map plot
map_plot_ideal = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_ideal

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)